In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
import torchvision
from torchinfo import summary
from tqdm import tqdm
from nn_zoo.datamodules import MNISTDataModule
from nn_zoo.models.components import DepthwiseSeparableConv2d
import matplotlib.pyplot as plt

In [ ]:
dm = MNISTDataModule(
        data_dir="../../../data",
        dataset_params={
            "download": True,
            "transform": torchvision.transforms.Compose(
                [
                    torchvision.transforms.Resize((32, 32)),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.RandomRotation(30),
                ]
            ),
        },
        loader_params={
            "batch_size": 128
        },
        qmnist=True
    )

dm.setup()

train_loader = dm.train_dataloader()
val_loader = dm.val_dataloader()

In [ ]:
x, y = next(iter(val_loader))

plt.figure(figsize=(20, 5))
plt.imshow(torchvision.utils.make_grid(x, 32).permute(1, 2, 0))
plt.title("Images")
plt.axis("off")
plt.show()

In [ ]:
class LPIPS(nn.Module):
    def __init__(self):
        super(LPIPS, self).__init__()
        self.conv1 = DepthwiseSeparableConv2d(1, 4, 3, 2, 1)
        self.conv2 = DepthwiseSeparableConv2d(4, 8, 3, 2, 1)
        self.conv3 = DepthwiseSeparableConv2d(8, 16, 3, 2, 1)
        self.conv4 = DepthwiseSeparableConv2d(16, 32, 3, 2, 1)
        self.conv5 = DepthwiseSeparableConv2d(32, 64, 3, 2, 1)

        self.fc1 = nn.Linear(64, 10)

        # self.load_state_dict(torch.load("model.pth"))

    def forward(self, x, lpips=False):
        x1 = F.relu(self.conv1(x))
        x2 = F.relu(self.conv2(x1))
        x3 = F.relu(self.conv3(x2))
        x4 = F.relu(self.conv4(x3))
        x5 = F.relu(self.conv5(x4))

        x5 = x5.view(x5.size(0), -1)

        x = self.fc1(x5)

        if lpips:
            return x, x1, x2, x3, x4
        
        return x
        
    @torch.no_grad()
    @torch.jit.export
    def lpips(self, img1, img2, linear=False):
        # Forward pass with lpips=True to get feature maps
        features1 = self.forward(img1, lpips=True)
        features2 = self.forward(img2, lpips=True)

        # Calculate the LPIPS score
        if linear:
            scores = list(map(lambda x, y: F.mse_loss(x, y), features1, features2))
        else:
            scores = list(map(lambda x, y: F.mse_loss(x, y), features1[3:], features2[3:]))

        # Sum the distances to get the LPIPS score
        lpips_score = sum(scores)

        return lpips_score
    
model = Model()
model = model.to("mps")
print(summary(model, input_size=(1, 1, 32, 32), device="mps", depth=1))

In [ ]:
from ema_pytorch import EMA

ema = EMA(model, beta=0.999, update_after_step=100, update_every=10)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
@torch.no_grad()
def eval(model, loader):
    model.eval()
    correct = 0
    total = 0
    loss = 0
    for x, y in loader:
        x, y = x.to("mps"), y.to("mps")
        y_hat = model(x)
        pred = y_hat.argmax(dim=1, keepdim=True)
        correct += pred.eq(y.view_as(pred)).sum().item()
        total += len(x)
        loss += F.cross_entropy(y_hat, y).item()
    return correct / total, loss / len(loader)

In [ ]:
if 'roll_loss' not in locals():
    roll_loss = 0

for epoch in range(10):
    model.train()
    val_acc, val_loss = eval(model, val_loader)
    lpips = model.lpips(torch.randn(2048, 1, 32, 32, device="mps"), torch.randn(2048, 1, 32, 32, device="mps")).item()
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}", ncols=150)
    for x, y in pbar:
        x, y = x.to("mps"), y.to("mps")
        optimizer.zero_grad()
        y_hat = model(x)
        loss = F.cross_entropy(y_hat, y)
        loss.backward()
        optimizer.step()

        ema.update()

        roll_loss = 0.99 * roll_loss + 0.01 * loss.item()
        pbar.set_postfix_str(f"loss: {roll_loss:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.2%}, lpips: {lpips:.4f}")
    

In [ ]:
torch.save(model.state_dict(), "model1.pth")

In [ ]:
# plot the distribution of weights for each layer
plt.subplot()